**Data Loading Section**

In [1]:
import os
import gc
from glob import glob
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import lightgbm as lgb

from sklearn.model_selection import StratifiedGroupKFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import exploratory_data_analysis as data_nb
from exploratory_data_analysis import Aggregator

data_nb.Aggregator.group_aggregators = [pl.n_unique] #pl.max, pl.min, 
# data_nb.Aggregator.str_aggregators = [pl.max, pl.min, pl.first, pl.last, pl.n_unique]
# train_df = data_nb.prepare_df(data_nb.CFG.train_dir)
base_files = [
    "_static_cb_0.parquet",
    "_static_0_*.parquet",
    "_applprev_1_*.parquet",
    "_tax_registry_a_1.parquet",
    "_tax_registry_b_1.parquet",
    "_tax_registry_c_1.parquet",
    "_other_1.parquet",
    "_person_1.parquet",
    "_deposit_1.parquet",
    "_debitcard_1.parquet",
    "_credit_bureau_b_1.parquet",
    "_credit_bureau_b_2.parquet",
]
base_agg = Aggregator(
    num_aggregators = [pl.max, pl.min, pl.first, pl.last, pl.mean],
    str_aggregators = [pl.max, pl.min, pl.first, pl.last],
    group_aggregators = [pl.max, pl.min, pl.first, pl.last],
    str_mode = True
)
breakpoint()
train_df = data_nb.prepare_df(base_files, data_nb.CFG.train_dir, base_agg)

# train_df = pd.read_parquet('/kaggle/usr/lib/home_credit_baseline_data/train_full.parquet')
cat_cols = list(train_df.select_dtypes("category").columns)


  files:  train_base.parquet
  chunk:  0
  files:  _static_cb_0.parquet (depth: 0)
  chunk:  0
  files:  _static_0_*.parquet (depth: 0)
  chunk:  0
  chunk:  1
  files:  _applprev_1_*.parquet (depth: 1)
  chunk:  0
  chunk:  1
  agg, depth 1
  files:  _tax_registry_a_1.parquet (depth: 1)
  chunk:  0
  agg, depth 1
  files:  _tax_registry_b_1.parquet (depth: 1)
  chunk:  0
  agg, depth 1
  files:  _tax_registry_c_1.parquet (depth: 1)
  chunk:  0
  agg, depth 1
  files:  _other_1.parquet (depth: 1)
  chunk:  0
  agg, depth 1
  files:  _person_1.parquet (depth: 1)
  chunk:  0
  agg, depth 1
  files:  _deposit_1.parquet (depth: 1)
  chunk:  0
  agg, depth 1
  files:  _debitcard_1.parquet (depth: 1)
  chunk:  0
  agg, depth 1
  files:  _credit_bureau_b_1.parquet (depth: 1)
  chunk:  0
  agg, depth 1
  files:  _credit_bureau_b_2.parquet (depth: 2)
  chunk:  0
  agg, depth 2
  feats_df shape:	 (1526659, 927)
Feature Engineering...
Filter cols...
  feats_df shape:	 (1526659, 517)
Convert to p

In [2]:
train_df.shape

(1526659, 517)

In [3]:
train_df.describe()

,case_id,WEEK_NUM,target,month_decision,weekday_decision,assignmentdate_238D,assignmentdate_4527235D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,firstquarter_103L,fourthquarter_440L,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtcount_4527229L,pmtcount_693L,pmtscount_423L,pmtssum_45A,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,secondquarter_766L,thirdquarter_1082L,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credamount_770A,currdebt_22A,currdebtcredtyperange_828A,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursedcredamount_1113A,downpmt_116A,dtlastpmtallstes_4499206D,eir_270L,firstclxcampaign_1125D,firstdatedue_489D,homephncnt_628L,inittransactionamount_650A,interestrate_311L,lastactivateddate_801D,lastapplicationdate_877D,lastapprcredamount_781A,lastapprdate_640D,lastdelinqdate_224D,lastrejectcredamount_222A,lastrejectdate_50D,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstldate_3546855D,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P,mindbdtollast24m_4525191P,mobilephncnt_593L,monthsannuity_845L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numincomingpmts_3546848L,numinstlallpaidearly3d_817L,numinstls_657L,numinstlsallpaid_934L,numinstlswithdpd10_728L,numinstlswithdpd5_4187116L,numinstlswithoutdpd_562L,numinstmatpaidtearly2d_4499204L,numinstpaid_4499208L,numinstpaidearly3d_3546850L,numinstpaidearly3dest_4493216L,numinstpaidearly5d_1087L,numinstpaidearly5dest_4493211L,numinstpaidearly5dobd_4499205L,numinstpaidearly_338L,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,validfrom_1069D,max_actualdpd_943P,max_annuity_853A,max_credacc_actualbalance_314A,max_credacc_credlmt_575A,max_credacc_maxhisbal_375A,max_credacc_minhisbal_90A,max_credamount_590A,max_currdebt_94A,max_downpmt_134A,max_mainoccupationinc_437A,max_maxdpdtolerance_577P,max_outstandingdebt_522A,max_revolvingaccount_394A,min_actualdpd_943P,min_annuity_853A,min_credacc_actualbalance_314A,min_credacc_credlmt_575A,min_credacc_maxhisbal_375A,min_credacc_minhisbal_90A,min_credamount_590A,min_c

**Sensitive Features to Test**

In [4]:
train_df['first_sex_738L']

0          F
1          M
2          F
3          F
4          F
          ..
1526654    F
1526655    F
1526656    M
1526657    F
1526658    F
Name: first_sex_738L, Length: 1526659, dtype: category
Categories (2, object): ['F', 'M']

In [5]:
train_df['first_language1_981M']

0            P10_39_147
1            P10_39_147
2            P10_39_147
3            P10_39_147
4            P10_39_147
               ...     
1526654    P209_127_106
1526655    P209_127_106
1526656    P209_127_106
1526657    P209_127_106
1526658      P10_39_147
Name: first_language1_981M, Length: 1526659, dtype: category
Categories (3, object): ['P10_39_147', 'P209_127_106', 'a55475b1']

In [6]:
train_df['maritalst_385M']

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
             ...   
1526654    a55475b1
1526655    a55475b1
1526656    a55475b1
1526657    3439d993
1526658    b6cabe76
Name: maritalst_385M, Length: 1526659, dtype: category
Categories (6, object): ['3439d993', '38c061ee', 'a55475b1', 'a7fcb6e5', 'b6cabe76', 'ecd83604']

In [7]:
train_df['education_1103M']

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
             ...   
1526654    a55475b1
1526655    a55475b1
1526656    a55475b1
1526657    6b2ae0fa
1526658    a55475b1
Name: education_1103M, Length: 1526659, dtype: category
Categories (5, object): ['39a0853f', '6b2ae0fa', '717ddd49', 'a55475b1', 'c8e1a1d0']

In [8]:
train_df['dateofbirth_337D']

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
            ...   
1526654   -22193.0
1526655   -25541.0
1526656   -15771.0
1526657   -25814.0
1526658   -26485.0
Name: dateofbirth_337D, Length: 1526659, dtype: float64

Class Imbalance Analysis

In [9]:
np.unique(train_df['target'], return_counts=True)

(array([0, 1]), array([1478665,   47994]))

In [10]:
np.unique(train_df['min_sex_738L'], return_counts=True)

(array(['F', 'M'], dtype=object), array([952776, 573883]))

In [11]:
1478665/(1578665+47994)

0.9090196531663981

**Training Section**

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train_df, valid_df = train_test_split(train_df, test_size=0.1, random_state=42) #Need Validation set for ADS analysis

In [14]:
y_valid = valid_df['target']

In [82]:
def gini_stability(base, score_col="score", w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", score_col]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", score_col]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x[score_col])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

In [16]:
%%time
X = train_df.drop(columns=["target", "case_id", "WEEK_NUM"])
print("X shape: ", X.shape)
y = train_df["target"]
weeks = train_df["WEEK_NUM"]

cv = StratifiedGroupKFold(n_splits=5, shuffle=False)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 8,
    "max_bin": 255,
    "learning_rate": 0.05,
    "n_estimators": 1000,
    "colsample_bytree": 0.8, 
    "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 42,
    "device": "gpu",
}

fitted_models = []
oof_pred = np.zeros(X.shape[0])

for idx_train, idx_valid in cv.split(X, y, groups=weeks):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[lgb.log_evaluation(100), lgb.early_stopping(100)]
    )
    fitted_models.append(model)
    val_pred = model.predict_proba(X_valid)[:, 1]
    oof_pred[idx_valid] = val_pred
    gc.collect()

X shape:  (1373993, 514)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.818356
[200]	valid_0's auc: 0.827496
[300]	valid_0's auc: 0.830195
[400]	valid_0's auc: 0.831277
[500]	valid_0's auc: 0.831699
[600]	valid_0's auc: 0.831875
[700]	valid_0's auc: 0.832015
[800]	valid_0's auc: 0.832212
[900]	valid_0's auc: 0.832416
[1000]	valid_0's auc: 0.832558
Did not meet early stopping. Best iteration is:
[974]	valid_0's auc: 0.832624
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.825715
[200]	valid_0's auc: 0.834423
[300]	valid_0's auc: 0.837212
[400]	valid_0's auc: 0.838332
[500]	valid_0's auc: 0.838625
Early stopping, best iteration is:
[469]	valid_0's auc: 0.838685
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.820924
[200]	valid_0's auc: 0.829065
[300]	valid_0's auc: 0.832068
[400]	valid_0's auc: 0.833192
[500]	valid_0's auc: 0.833583
[600]	valid_0's auc: 0.833802
[700]	valid_0's

**Accuracy Measures**

In [17]:
oof_models_dict = [(str(i), model) for i, model in enumerate(fitted_models)]

model = VotingClassifier(
    estimators=oof_models_dict,
    voting='soft',
)
model.estimators_ = fitted_models
model.le_ = LabelEncoder().fit(y)
model.classes_ = model.le_.classes_

In [18]:
valid_preds = model.predict(valid_df.drop(columns=["target", "case_id", "WEEK_NUM"]))
print(valid_preds.shape)
print(valid_df.shape)
valid_preds_proba = model.predict_proba(valid_df.drop(columns=["target", "case_id", "WEEK_NUM"]))[:,1]

(152666,)
(152666, 517)


In [19]:
np.unique(np.round(valid_preds), return_counts=True)

(array([0, 1]), array([152561,    105]))

In [20]:
np.unique(np.round(valid_preds_proba), return_counts=True)

(array([0., 1.]), array([152561,    105]))

In [24]:
y_valid = valid_df['target']
print(y_valid.shape)

(152666,)


In [25]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
print("Accuracy: ", accuracy_score(y_valid, np.round(valid_preds)))
print("Recall: ", recall_score(y_valid, np.round(valid_preds)))
print("Precision: ", precision_score(y_valid, np.round(valid_preds)))
print("F1 Score: ", f1_score(y_valid, np.round(valid_preds)))

Accuracy:  0.9692203896086883
Recall:  0.011899702507437314
Precision:  0.5333333333333333
F1 Score:  0.02327998337144045


In [26]:
oof_df = valid_df[["WEEK_NUM", "target"]].copy()
oof_df["pred_oof"] = valid_preds_proba
gini_score = gini_stability(oof_df, score_col="pred_oof")
print("gini_score:\t", gini_score)

gini_score:	 0.6405826780625778


In [27]:
roc_auc_oof = roc_auc_score(y_valid, valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)

CV roc_auc_oof:  0.8378891642875189


In [28]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_valid, np.round(valid_preds)).ravel()
print("Confusion Matrix: ", confusion_matrix(y_valid, np.round(valid_preds)))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Confusion Matrix:  [[147911     49]
 [  4650     56]]
FPR:  0.0003311705866450392
FNR:  0.9881002974925627


In [61]:
male_idx = valid_df.drop(valid_df[valid_df['first_sex_738L'] != "M"].index)
female_idx = valid_df.drop(valid_df[valid_df['first_sex_738L'] != 'F'].index)
language_idx_1 = valid_df.drop(valid_df[valid_df['first_language1_981M'] != 'P10_39_147'].index)
language_idx_2 = valid_df.drop(valid_df[valid_df['first_language1_981M'] != 'P209_127_106'].index)
language_idx_other = valid_df.drop(valid_df[valid_df['first_language1_981M'] != 'a55475b1'].index)
marital_idx_1 = valid_df.drop(valid_df[valid_df['maritalst_385M'] != '3439d993'].index)
marital_idx_2 = valid_df.drop(valid_df[valid_df['maritalst_385M'] != '38c061ee'].index)
marital_idx_3 = valid_df.drop(valid_df[valid_df['maritalst_385M'] != 'a7fcb6e5'].index)
marital_idx_4 = valid_df.drop(valid_df[valid_df['maritalst_385M'] != 'b6cabe76'].index)
marital_idx_5 = valid_df.drop(valid_df[valid_df['maritalst_385M'] != 'ecd83604'].index)
marital_idx_other = valid_df.drop(valid_df[valid_df['maritalst_385M'] != 'a55475b1'].index)
education_idx_1 = valid_df.drop(valid_df[valid_df['education_1103M'] != '39a0853f'].index)
education_idx_2 = valid_df.drop(valid_df[valid_df['education_1103M'] != '6b2ae0fa'].index)
education_idx_3 = valid_df.drop(valid_df[valid_df['education_1103M'] != '717ddd49'].index)
education_idx_4 = valid_df.drop(valid_df[valid_df['education_1103M'] != 'c8e1a1d0'].index)
education_idx_other = valid_df.drop(valid_df[valid_df['education_1103M'] != 'a55475b1'].index)
# dob_lt_30 = valid_df.drop(valid_df[valid_df['dateofbirth_337D'].abs()/365 < 30)
# dob_30_40 = valid_df.drop(valid_df[(valid_df['dateofbirth_337D'].abs()/365 < 40) & (valid_df['dateofbirth_337D'].abs()/365 >= 30))
# dob_40_50 = valid_df.drop(valid_df[(valid_df['dateofbirth_337D'].abs()/365 < 50) && (valid_df['dateofbirth_337D'].abs()/365 >= 40))
# dob_50_60 = valid_df.drop(valid_df[(valid_df['dateofbirth_337D'].abs()/365 < 60) && (valid_df['dateofbirth_337D'].abs()/365 >= 50))
# dob_gt_60 = valid_df.where(valid_df['dateofbirth_337D'].abs()/365 >= 60)

# Categorize age into bins and combine with sex
valid_df['age_bucket'] = pd.cut(valid_df['dateofbirth_337D'].abs()/365, bins=[0, 30, 40, 50, 60, 200], labels=['<30', '30-40', '40-50', '50-60', '>60'], right=False)
valid_df['group_age_sex'] = valid_df['first_sex_738L'].astype(str) + ' & ' + valid_df['age_bucket'].astype(str)

# Dictionary to hold each group's DataFrame
group_dfs = {}

# Iterate over each unique group and create a DataFrame
for group in valid_df['group_age_sex'].unique():
    group_dfs[group] = valid_df[valid_df['group_age_sex'] == group]

In [90]:
#Male Section
print("Male Section")
new_idx_df = male_idx
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Male Section
Accuracy:  0.962094649990406
Recall:  0.015610651974288337
Precision:  0.5396825396825397
F1 Score:  0.03034359660865685
gini_score:	 0.5861827115191606
CV roc_auc_oof:  0.8218132941493952
Confusion Matrix: 
 [[55120    29]
 [ 2144    34]]
FPR:  0.0005258481568115469
FNR:  0.9843893480257117


In [91]:
print("Female Section")
new_idx_df = female_idx
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Female Section
Accuracy:  0.9735952973574115
Recall:  0.008817635270541082
Precision:  0.5238095238095238
F1 Score:  0.017343318880567594
Gini Score code error.
CV roc_auc_oof:  0.8453865912852686
Confusion Matrix: 
 [[91900    20]
 [ 2473    22]]
FPR:  0.0002175805047867711
FNR:  0.9911823647294589


In [92]:
print("Language 1 Section")
new_idx_df = language_idx_1
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Language 1 Section
Accuracy:  0.9663869518969389
Recall:  0.013014421385860007
Precision:  0.49333333333333335
F1 Score:  0.025359835503769707
gini_score:	 0.6263561039214064
CV roc_auc_oof:  0.8296546847052142
Confusion Matrix: 
 [[81729    38]
 [ 2806    37]]
FPR:  0.0004647351621069625
FNR:  0.98698557861414


In [93]:
print("Language 2 Section")
new_idx_df = language_idx_2
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Language 2 Section
Accuracy:  0.9725146485171947
Recall:  0.008293838862559242
Precision:  0.5833333333333334
F1 Score:  0.016355140186915893
Gini Score code error.
CV roc_auc_oof:  0.8443144447667762
Confusion Matrix: 
 [[59571    10]
 [ 1674    14]]
FPR:  0.00016783874053809101
FNR:  0.9917061611374408


In [94]:
print("Language Other Section")
new_idx_df = language_idx_other
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Language Other Section
Accuracy:  0.9748047738323266
Recall:  0.02857142857142857
Precision:  0.8333333333333334
F1 Score:  0.05524861878453038
Gini Score code error.
CV roc_auc_oof:  0.8769726039236021
Confusion Matrix: 
 [[6611    1]
 [ 170    5]]
FPR:  0.00015124016938898973
FNR:  0.9714285714285714


In [95]:
print("Marital 1 Section")
new_idx_df = marital_idx_1
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Marital 1 Section
Accuracy:  0.9719685554002288
Recall:  0.012281835811247576
Precision:  0.5428571428571428
F1 Score:  0.02402022756005057
Gini Score code error.
CV roc_auc_oof:  0.8471274856793461
Confusion Matrix: 
 [[53518    16]
 [ 1528    19]]
FPR:  0.0002988754810027272
FNR:  0.9877181641887525


In [96]:
print("Marital 2 Section")
new_idx_df = marital_idx_2
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Marital 2 Section
Accuracy:  0.9662878787878788
Recall:  0.0
Precision:  0.0
F1 Score:  0.0
Gini Score code error.
CV roc_auc_oof:  0.838037011969222
Confusion Matrix: 
 [[2551    1]
 [  88    0]]
FPR:  0.00039184952978056425
FNR:  1.0


In [97]:
print("Marital 3 Section")
new_idx_df = marital_idx_3
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Marital 3 Section
Accuracy:  0.9642430278884462
Recall:  0.009831460674157303
Precision:  0.35
F1 Score:  0.01912568306010929
Gini Score code error.
CV roc_auc_oof:  0.8393468869071653
Confusion Matrix: 
 [[19355    13]
 [  705     7]]
FPR:  0.00067121024370095
FNR:  0.9901685393258427


In [98]:
print("Marital 4 Section")
new_idx_df = marital_idx_4
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Marital 4 Section
Accuracy:  0.9739518287243533
Recall:  0.020134228187919462
Precision:  1.0
F1 Score:  0.03947368421052631
Gini Score code error.
CV roc_auc_oof:  0.8659661674112854
Confusion Matrix: 
 [[5456    0]
 [ 146    3]]
FPR:  0.0
FNR:  0.9798657718120806


In [99]:
print("Marital 5 Section")
new_idx_df = marital_idx_5
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Marital 5 Section
Accuracy:  0.9408163265306122
Recall:  0.0
Precision:  0.0
F1 Score:  0.0
Gini Score code error.
CV roc_auc_oof:  0.7990874410950707
Confusion Matrix: 
 [[461   0]
 [ 29   0]]
FPR:  0.0
FNR:  1.0


/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [100]:
print("Marital Other Section")
new_idx_df = marital_idx_other
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Marital Other Section
Accuracy:  0.9681563317061375
Recall:  0.01276595744680851
Precision:  0.5869565217391305
F1 Score:  0.024988431281813973
Gini Score code error.
CV roc_auc_oof:  0.8279824430475298
Confusion Matrix: 
 [[64033    19]
 [ 2088    27]]
FPR:  0.0002966339848872791
FNR:  0.9872340425531915


In [101]:
print("Education 1 Section")
new_idx_df = education_idx_1
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Education 1 Section
Accuracy:  0.9450503355704698
Recall:  0.03018867924528302
Precision:  0.6153846153846154
F1 Score:  0.05755395683453238
Gini Score code error.
CV roc_auc_oof:  0.8127110228401192
Confusion Matrix: 
 [[4498    5]
 [ 257    8]]
FPR:  0.0011103708638685321
FNR:  0.969811320754717


In [102]:
print("Education 2 Section")
new_idx_df = education_idx_2
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Education 2 Section
Accuracy:  0.9738648654614485
Recall:  0.007633587786259542
Precision:  0.391304347826087
F1 Score:  0.01497504159733777
Gini Score code error.
CV roc_auc_oof:  0.8571215448113725
Confusion Matrix: 
 [[44110    14]
 [ 1170     9]]
FPR:  0.0003172876439126099
FNR:  0.9923664122137404


In [103]:
print("Education 3 Section")
new_idx_df = education_idx_3
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Education 3 Section
Accuracy:  0.9677895981087471
Recall:  0.013761467889908258
Precision:  0.5
F1 Score:  0.026785714285714284
Gini Score code error.
CV roc_auc_oof:  0.8361485398137125
Confusion Matrix: 
 [[13094     6]
 [  430     6]]
FPR:  0.0004580152671755725
FNR:  0.9862385321100917


In [104]:
print("Education 4 Section")
new_idx_df = education_idx_4
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Education 4 Section
Accuracy:  0.9584837545126353
Recall:  0.0
Precision:  0.0
F1 Score:  0.0
Gini Score code error.
CV roc_auc_oof:  0.8273151559813314
Confusion Matrix: 
 [[531   0]
 [ 23   0]]
FPR:  0.0
FNR:  1.0


/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [105]:
print("Education Other Section")
new_idx_df = education_idx_other
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Education Other Section
Accuracy:  0.968242881423017
Recall:  0.01205699671172817
Precision:  0.5789473684210527
F1 Score:  0.023622047244094484
Gini Score code error.
CV roc_auc_oof:  0.8284560226344831
Confusion Matrix: 
 [[83141    24]
 [ 2704    33]]
FPR:  0.00028858293753381833
FNR:  0.9879430032882718


In [108]:
for key in group_dfs:
    print(key)
    print(group_dfs[key].shape)

F & >60
(18657, 519)
F & 30-40
(20770, 519)
M & 50-60
(7293, 519)
F & <30
(12360, 519)
F & 40-50
(19041, 519)
M & >60
(6862, 519)
F & 50-60
(15308, 519)
M & 40-50
(11377, 519)
F & nan
(8279, 519)
M & <30
(10944, 519)
M & 30-40
(15213, 519)
M & nan
(5638, 519)
nan & 40-50
(156, 519)
nan & 30-40
(371, 519)
nan & <30
(170, 519)
nan & nan
(72, 519)
nan & 50-60
(91, 519)
nan & >60
(64, 519)


In [110]:
#We pick the two most populus combinations for male and female each.
combo_1_f = group_dfs['F & 30-40']
combo_2_f = group_dfs['F & 40-50']
combo_1_m = group_dfs['M & 30-40']
combo_2_m = group_dfs['M & 40-50']

In [111]:
print("Female 30-40 Section")
new_idx_df = combo_1_f
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Female 30-40 Section
Accuracy:  0.970341839191141
Recall:  0.011363636363636364
Precision:  0.5
F1 Score:  0.022222222222222223
Gini Score code error.
CV roc_auc_oof:  0.8355586496960418
Confusion Matrix: 
 [[20147     7]
 [  609     7]]
FPR:  0.0003473255929344051
FNR:  0.9886363636363636


In [112]:
print("Female 40-50 Section")
new_idx_df = combo_2_f
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Female 40-50 Section
Accuracy:  0.9730581377028518
Recall:  0.015473887814313346
Precision:  0.6666666666666666
F1 Score:  0.030245746691871456
Gini Score code error.
CV roc_auc_oof:  0.8590597299253581
Confusion Matrix: 
 [[18520     4]
 [  509     8]]
FPR:  0.00021593608291945585
FNR:  0.9845261121856866


In [113]:
print("Male 30-40 Section")
new_idx_df = combo_1_m
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Male 30-40 Section
Accuracy:  0.9619404456714652
Recall:  0.01896551724137931
Precision:  0.5238095238095238
F1 Score:  0.036605657237936774
Gini Score code error.
CV roc_auc_oof:  0.8215932575637964
Confusion Matrix: 
 [[14623    10]
 [  569    11]]
FPR:  0.0006833868653044489
FNR:  0.9810344827586207


In [114]:
print("Male 40-50 Section")
new_idx_df = combo_2_m
new_y_valid = new_idx_df['target']
new_valid_preds = model.predict(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))
new_valid_preds_proba = model.predict_proba(new_idx_df.drop(columns=["target", "case_id", "WEEK_NUM", 'age_bucket', 'group_age_sex']))[:,1]
print("Accuracy: ", accuracy_score(new_y_valid, new_valid_preds))
print("Recall: ", recall_score(new_y_valid, new_valid_preds))
print("Precision: ", precision_score(new_y_valid, new_valid_preds))
print("F1 Score: ", f1_score(new_y_valid, new_valid_preds))
new_oof_df = new_idx_df[["WEEK_NUM", "target"]].copy()
new_oof_df["pred_oof"] = new_valid_preds_proba
try:
    gini_score = gini_stability(new_oof_df, score_col="pred_oof")
    print("gini_score:\t", gini_score)
except:
    print("Gini Score code error.")
roc_auc_oof = roc_auc_score(new_y_valid, new_valid_preds_proba)
print("CV roc_auc_oof: ", roc_auc_oof)
tn, fp, fn, tp = confusion_matrix(new_y_valid, new_valid_preds).ravel()
print("Confusion Matrix: \n", confusion_matrix(new_y_valid, new_valid_preds))
print("FPR: ", fp/(fp+tn))
print("FNR: ", fn/(tp+fn))

Male 40-50 Section
Accuracy:  0.9632592071723654
Recall:  0.007194244604316547
Precision:  0.42857142857142855
F1 Score:  0.014150943396226415
Gini Score code error.
CV roc_auc_oof:  0.8308932416110906
Confusion Matrix: 
 [[10956     4]
 [  414     3]]
FPR:  0.000364963503649635
FNR:  0.9928057553956835
